In [1]:
! pip install fuzzywuzzy[speedup]

In [1]:
import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import sklearn.metrics
import swifter
from sklearn import metrics

import fastai
from fastai.callbacks import *
from fastai.datasets import *
from fastai.imports import nn, torch
from fastai.metrics import *
from fastai.text import *
from fastai.train import *
from fastai.vision import *
from news_utils.clean.english import clean

torch.cuda.set_device(2)

pd.options.display.max_colwidth = 1000

In [2]:
max_vocab = 30000
exp_folder = str(max_vocab) + '_threads_last'

IN = Path('~/data/ynacc_proc/replicate/threads_last')
EX_PA = Path('/mnt/data/group07/johannes/ynacc_proc/replicate/' + exp_folder)
exp_folder

'30000_threads_last'

In [3]:
df_train = pd.read_csv(IN/'train.csv')
df_train

,text
0,"xx_thread_start xx_comment_start xx_last She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_thread_end"
1,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start xx_last Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_thread_end"
2,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_comment_start xx_last Yeah, because #$%$ feelings and human nature, that is all deranged. xx_comment_end xx_thread_end"
3,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_comment_start Yeah, because #$%$ feelings and human nature, that is all deranged. xx_comment_end xx_comment_start xx_last are you serious no its not ok to bang a student if shes in highschool and your her teacher no matter what her age is xx_comment_end xx_thread_end"
4,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_comment_start Yeah, because #$%$ feelings and human nature, that is all deranged. xx_comment_end xx_comment_start are you serious no its not ok to bang a student if shes in highschool and your her teacher no matter what her age is xx_comment_end xx_comment_start xx_last Oblivion, you are not the brightest bulb in the room. A) The incident occurred in 0000 which would make the student 00 or 00 at the time of the incident not 00 and the teacher 00 or 00. B) It is and should be against the law for high school teachers to sleep with their students...."
5,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_comment_start Yeah, because #$%$ feelings and human nature, that is all deranged. xx_comment_end xx_comment_start are you serious no its not ok to bang a student if shes in highschool and your her teacher no matter what her age is xx_comment_end xx_comment_start Oblivion, you are not the brightest bulb in the room. A) The incident occurred in 0000 which would make the student 00 or 00 at the time of the incident not 00 and the teacher 00 o

In [4]:
df_val = pd.read_csv(IN/'val.csv')
df_val

,text
0,"xx_thread_start xx_comment_start xx_last A monster child ivanka would be proud of her monster hitler dad drumpf. That is common sense. Hitler only had his mistress that followed him to their destructive end. This evil family cannot see past their noses to see any difference. Jonathan sticked to Evil Saul leadership to death even though Saul tried to kill David that would later be King of Israel. Her husband believes in an "" eye for an eye ""because he is jewish & drumpf thinks the same way . PALISTINES LIVE IN RUBBLE because of that same mentality so yes, hitler drumpf would destroy America trying to make it great again along with his evil daughter. Chelsea should consider making better friends with stronger God-fearing people. Ivanka would not step in if she was waterboarded. Germans didn't intervene hitlers tortures nor did his mistress. xx_comment_end xx_thread_end"
1,"xx_thread_start xx_comment_start A monster child ivanka would be proud of her monster hitler dad drumpf. That is common sense. Hitler only had his mistress that followed him to their destructive end. This evil family cannot see past their noses to see any difference. Jonathan sticked to Evil Saul leadership to death even though Saul tried to kill David that would later be King of Israel. Her husband believes in an "" eye for an eye ""because he is jewish & drumpf thinks the same way . PALISTINES LIVE IN RUBBLE because of that same mentality so yes, hitler drumpf would destroy America trying to make it great again along with his evil daughter. Chelsea should consider making better friends with stronger God-fearing people. Ivanka would not step in if she was waterboarded. Germans didn't intervene hitlers tortures nor did his mistress. xx_comment_end xx_comment_start xx_last Wow mary, project much? xx_comment_end xx_thread_end"
2,"xx_thread_start xx_comment_start A monster child ivanka would be proud of her monster hitler dad drumpf. That is common sense. Hitler only had his mistress that followed him to their destructive end. This evil family cannot see past their noses to see any difference. Jonathan sticked to Evil Saul leadership to death even though Saul tried to kill David that would later be King of Israel. Her husband believes in an "" eye for an eye ""because he is jewish & drumpf thinks the same way . PALISTINES LIVE IN RUBBLE because of that same mentality so yes, hitler drumpf would destroy America trying to make it great again along with his evil daughter. Chelsea should consider making better friends with stronger God-fearing people. Ivanka would not step in if she was waterboarded. Germans didn't intervene hitlers tortures nor did his mistress. xx_comment_end xx_comment_start Wow mary, project much? xx_comment_end xx_comment_start xx_last What a bunch of garbage, weirdo. xx_comment_end xx_threa..."
3,"xx_thread_start xx_comment_start A monster child ivanka would be proud of her monster hitler dad drumpf. That is common sense. Hitler only had his mistress that followed him to their destructive end. This evil family cannot see past their noses to see any difference. Jonathan sticked to Evil Saul leadership to death even though Saul tried to kill David that would later be King of Israel. Her husband believes in an "" eye for an eye ""because he is jewish & drumpf thinks the same way . PALISTINES LIVE IN RUBBLE because of that same mentality so yes, hitler drumpf would destroy America trying to make it great again along with his evil daughter. Chelsea should consider making better friends with stronger God-fearing people. Ivanka would not step in if she was waterboarded. Germans didn't intervene hitlers tortures nor did his mistress. xx_comment_end xx_comment_start Wow mary, project much? xx_comment_end xx_comment_start What a bunch of garbage, weirdo. xx_comment_end xx_comment_start ..."
4,xx_thread_start xx_comment_start xx_last Rumors are spread by the press. Don't ever listen to opinions of the press. The only time I listen is when the c

In [5]:
df_train

,text
0,"xx_thread_start xx_comment_start xx_last She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_thread_end"
1,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start xx_last Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_thread_end"
2,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_comment_start xx_last Yeah, because #$%$ feelings and human nature, that is all deranged. xx_comment_end xx_thread_end"
3,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_comment_start Yeah, because #$%$ feelings and human nature, that is all deranged. xx_comment_end xx_comment_start xx_last are you serious no its not ok to bang a student if shes in highschool and your her teacher no matter what her age is xx_comment_end xx_thread_end"
4,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_comment_start Yeah, because #$%$ feelings and human nature, that is all deranged. xx_comment_end xx_comment_start are you serious no its not ok to bang a student if shes in highschool and your her teacher no matter what her age is xx_comment_end xx_comment_start xx_last Oblivion, you are not the brightest bulb in the room. A) The incident occurred in 0000 which would make the student 00 or 00 at the time of the incident not 00 and the teacher 00 or 00. B) It is and should be against the law for high school teachers to sleep with their students...."
5,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_comment_start Yeah, because #$%$ feelings and human nature, that is all deranged. xx_comment_end xx_comment_start are you serious no its not ok to bang a student if shes in highschool and your her teacher no matter what her age is xx_comment_end xx_comment_start Oblivion, you are not the brightest bulb in the room. A) The incident occurred in 0000 which would make the student 00 or 00 at the time of the incident not 00 and the teacher 00 o

In [6]:
df_train[df_train['text'].str.contains("00")]

,text
0,"xx_thread_start xx_comment_start xx_last She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_thread_end"
1,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start xx_last Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_thread_end"
2,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_comment_start xx_last Yeah, because #$%$ feelings and human nature, that is all deranged. xx_comment_end xx_thread_end"
3,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_comment_start Yeah, because #$%$ feelings and human nature, that is all deranged. xx_comment_end xx_comment_start xx_last are you serious no its not ok to bang a student if shes in highschool and your her teacher no matter what her age is xx_comment_end xx_thread_end"
4,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_comment_start Yeah, because #$%$ feelings and human nature, that is all deranged. xx_comment_end xx_comment_start are you serious no its not ok to bang a student if shes in highschool and your her teacher no matter what her age is xx_comment_end xx_comment_start xx_last Oblivion, you are not the brightest bulb in the room. A) The incident occurred in 0000 which would make the student 00 or 00 at the time of the incident not 00 and the teacher 00 or 00. B) It is and should be against the law for high school teachers to sleep with their students...."
5,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_comment_start Yeah, because #$%$ feelings and human nature, that is all deranged. xx_comment_end xx_comment_start are you serious no its not ok to bang a student if shes in highschool and your her teacher no matter what her age is xx_comment_end xx_comment_start Oblivion, you are not the brightest bulb in the room. A) The incident occurred in 0000 which would make the student 00 or 00 at the time of the incident not 00 and the teacher 00 o

In [7]:
# tok = Tokenizer()

In [8]:
# tokens = tok.process_all(df_train['text'])

In [9]:
# test_vocab = Vocab.create(sum(tokens, []))

In [10]:
data_lm = TextLMDataBunch.from_df(EX_PA, df_train, df_val, max_vocab=max_vocab, text_cols=0)

In [11]:
itos_new = data_lm.train_dl.vocab.itos

In [12]:
specials = ['xx_comment_start', 'xx_comment_end', 'xx_thread_start', 'xx_thread_end', 'xx_last']

In [13]:
for s in specials:
    itos_new.remove(s)
    itos_new.insert(2, s)

In [14]:
itos_new

['xxunk',
 'xxpad',
 'xx_last',
 'xx_thread_end',
 'xx_thread_start',
 'xx_comment_end',
 'xx_comment_start',
 'xxbos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 '.',
 'the',
 ',',
 'to',
 'and',
 'a',
 'of',
 'is',
 'you',
 'that',
 'i',
 'in',
 'it',
 '"',
 'are',
 'for',
 'not',
 'they',
 '?',
 "n't",
 "'s",
 'have',
 'do',
 'be',
 'with',
 'was',
 'this',
 'on',
 '!',
 'he',
 'as',
 'but',
 'or',
 '-',
 'people',
 'your',
 'if',
 'all',
 'what',
 'so',
 'we',
 'their',
 'who',
 'just',
 '0',
 'no',
 'would',
 'like',
 'will',
 'about',
 'she',
 'there',
 '00',
 'at',
 '...',
 'can',
 'one',
 'my',
 'from',
 'by',
 'has',
 'her',
 'them',
 'out',
 'an',
 'when',
 'his',
 'up',
 'get',
 ')',
 '$',
 'how',
 'because',
 'more',
 '4',
 'did',
 '(',
 'know',
 'were',
 'does',
 'me',
 'should',
 'had',
 'why',
 'think',
 '%',
 'then',
 'only',
 'than',
 'been',
 'other',
 "'",
 'time',
 'some',
 'any',
 'even',
 'our',
 'being',
 'these',
 'want',
 'right',
 'now',
 'him',
 'whit

In [15]:
vocab = Vocab(itos_new)

In [16]:
data_lm = TextLMDataBunch.from_df(EX_PA, df_train, df_val, vocab=vocab, text_cols=0)

In [17]:
data_lm.save()